In [1]:
import os, pandas as pd
# Add asset from file system

df_data_1 = pd.read_csv(os.environ['DSX_PROJECT_DIR']+'/datasets/Titanictrain.csv')
df_data_1.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
2,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
3,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
4,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S


In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_curve, auc, log_loss
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score 

In [3]:
df_train = pd.get_dummies(df_data_1, columns=["Pclass","Embarked","Sex"])
df_train.fillna(df_train["Age"].median(skipna=True), inplace=True)
df_train.drop('Sex_female', axis=1, inplace=True)
df_train.drop('PassengerId', axis=1, inplace=True)
df_train.drop('Name', axis=1, inplace=True)
df_train.drop('Ticket', axis=1, inplace=True)
df_train.drop('Cabin', axis=1, inplace=True)
df_train.head()

,Survived,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S,Sex_male
0,0,22.0,1,0,7.2500,0.0,0.0,1.0,0.0,0.0,1.0,1.0
1,0,35.0,0,0,8.0500,0.0,0.0,1.0,0.0,0.0,1.0,1.0
2,0,28.0,0,0,8.4583,0.0,0.0,1.0,0.0,1.0,0.0,1.0
3,0,54.0,0,0,51.8625,1.0,0.0,0.0,0.0,0.0,1.0,1.0
4,0,2.0,3,1,21.0750,0.0,0.0,1.0,0.0,0.0,1.0,1.0


In [4]:
X = df_train[["Age","SibSp","Parch","Fare","Pclass_1","Pclass_2","Pclass_3","Embarked_C","Embarked_Q","Embarked_S","Sex_male"]]
y = df_train['Survived']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

In [6]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
y_pred_proba = logreg.predict_proba(X_test)[:, 1]
[fpr, tpr, thr] = roc_curve(y_test, y_pred_proba)

In [7]:
print('Train/Test split results:')
print(logreg.__class__.__name__+" accuracy is %2.3f" % accuracy_score(y_test, y_pred))
print(logreg.__class__.__name__+" log_loss is %2.3f" % log_loss(y_test, y_pred_proba))
print(logreg.__class__.__name__+" auc is %2.3f" % auc(fpr, tpr))

Train/Test split results:
LogisticRegression accuracy is 0.811
LogisticRegression log_loss is 0.472
LogisticRegression auc is 0.830


In [8]:
from sklearn.externals import joblib
saved_model = joblib.dump(logreg, 'my_model.pkl')
print("<<<<<<<< ML MODEL CREATED AND SAVED >>>>>>>>>>>\n\n")

<<<<<<<< ML MODEL CREATED AND SAVED >>>>>>>>>>>




In [9]:
!python $DSX_PROJECT_DIR/scripts/deploy.py --username "gurvsin3" --model_path "my_model.pkl"

Lets Deploy your model...


2018-09-14 07:39:30,389 - watson_machine_learning_client.metanames - WARNING - 'AUTHOR_EMAIL' meta prop is deprecated. It will be ignored.
{'entity': {'author': {'name': 'gurvsin3'}, 'learning_iterations_url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/5b0ab00a-3bc8-4c6b-9ba2-c6362e633958/published_models/50ac5ed2-fe48-4d0f-938e-82915589931a/learning_iterations', 'deployments': {'url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/5b0ab00a-3bc8-4c6b-9ba2-c6362e633958/published_models/50ac5ed2-fe48-4d0f-938e-82915589931a/deployments', 'count': 0}, 'model_type': 'scikit-learn-0.19', 'latest_version': {'guid': 'f4564159-8520-434f-896b-f6bd260a6d7b', 'url': 'https://us-south.ml.cloud.ibm.com/v3/ml_assets/models/50ac5ed2-fe48-4d0f-938e-82915589931a/versions/f4564159-8520-434f-896b-f6bd260a6d7b', 'created_at': '2018-09-14T07:39:30.764Z'}, 'feedback_url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/5b0ab00a-3bc8-4c6b-9ba2-c6362e633958/publish